In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
filepath ='drive/MyDrive/hofstra/HitData/Hit215/ADV DA Project/Adv data project/Datasets/'
list_of_files = os.listdir(filepath)
print(list_of_files)

Mounted at /content/drive
['ADMISSIONS.csv', 'PRESCRIPTION.csv', 'heart_failure_clinical_records_dataset.csv', 'Heart_health.csv', 'cancer_classification.csv', 'diabetes_risk_prediction_dataset.csv', 'fetal_health.csv', 'pres.csv', 'Patient Admissions dummi data.csv', 'LAB.csv', 'LABEV.csv', 'DEEPSEPS FINAL DF.csv', 'los sepsis.csv', 'earlydiagseps.csv', 'diagseps.csv', 'labdss.csv', 'cardiac.csv', 'seps_diag.csv']


In [ ]:
df = pd.read_csv(filepath+ list_of_files[16] )

In [ ]:
df.head()

,subject_id,diagnosis,itemid,valuenum,valueuom,flag,label
0,10006,SEPSIS,50862,2,g/dL,abnormal,Albumin
1,10006,SEPSIS,50862,2,g/dL,abnormal,Albumin
2,10006,SEPSIS,50862,2,g/dL,abnormal,Albumin
3,10006,SEPSIS,50862,3,g/dL,NaN,Albumin
4,10006,SEPSIS,50862,3,g/dL,abnormal,Albumin


In [ ]:
flag_mapping = {"abnormal": 0, "": 1}

df["flag"] = df["flag"].replace(flag_mapping)
df['flag'] = df['flag'].fillna(1)

print(df.sample(5))

       subject_id                    diagnosis  itemid  valuenum valueuom  \
7648        41976                    PNEUMONIA   51277        15        %   
10629       41976         RESPIRATORY DISTRESS   51277        16        %   
10073       41976           UTI/PYELONEPHRITIS   51277        15        %   
6431        40310  AROMEGLEY;BURKITTS LYMPHOMA   51277        18        %   
6665        41976           UTI/PYELONEPHRITIS   51277        14        %   

      flag label  
7648     1   RDW  
10629    0   RDW  
10073    1   RDW  
6431     0   RDW  
6665     1   RDW  


In [ ]:
label_counts = df["label"].unique()
print(label_counts)

['Albumin' 'C-Reactive Protein' 'NTproBNP' 'Troponin I' 'Troponin T'
 'Lymphocytes' 'Fibrinogen, Functional' 'Neutrophils' 'Platelet Count'
 'RDW' 'WBC Count']


In [ ]:
cardiac_diagnoses = ['MITRAL REGURGITATION', 'CORONARY ARTERY DISEASE', 'CONGESTIVE HEART FAILURE',
                     'CRITICAL AORTIC STENOSIS', 'HYPOTENSION', 'INFERIOR MYOCARDIAL INFARCTION',
                     'CORONARY ARTERY BYPASS GRAFT', 'MI CHF', 'STEMI']

df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if any(cd in x for cd in cardiac_diagnoses) else 0)

df.sample(10)

,subject_id,diagnosis,itemid,valuenum,valueuom,flag,label
11901,44212,0,51265,59,K/uL,0,Platelet Count
7320,41976,0,51277,16,%,0,RDW
2153,10132,0,51265,483,K/uL,0,Platelet Count
6852,41976,0,51277,15,%,1,RDW
7869,42367,0,50862,2,g/dL,0,Albumin
9830,42302,0,51265,205,K/uL,1,Platelet Count
11955,44228,0,51265,449,K/uL,0,Platelet Count
11767,43881,0,51265,256,K/uL,1,Platelet Count
8109,41976,0,51256,89,%,0,Neutrophils
2204,40124,0,51265,449,K/uL,0,Platelet Count


In [ ]:
pivoted_data = df.pivot_table(index='subject_id', columns= 'label', values= 'valuenum', aggfunc='first')


pivoted_data.columns = [f"{col[1]}_{col[0]}" for col in pivoted_data.columns]


pivoted_data.reset_index(inplace=True)


pivoted_data = pivoted_data.merge(df[['subject_id','diagnosis']], on='subject_id')


pivoted_data.fillna(0, inplace=True)


print(pivoted_data.sample(1))

       subject_id  l_A  -_C  i_F  y_L      T_N   e_N    l_P   D_R  r_T  r_T  \
11187       42412  4.0  0.0  0.0  0.0  31877.0  90.0  127.0  19.0  0.0  0.0   

       B_W  diagnosis  
11187  0.0          0  


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
pivoted_data['diagnosis'] = le.fit_transform(pivoted_data['diagnosis'])


y = pivoted_data['diagnosis']
X = pivoted_data.iloc[:,:-1]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.30, random_state=20)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

from sklearn.metrics import f1_score, accuracy_score

acc = accuracy_score(y_test, y_pred)


print(f'KNN Accuracy: {acc}')

KNN Accuracy: 0.9473398479913138


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'DT F1: {f1}')
print(f'DT Accuracy: {acc}')



DT F1: 0.9430338500554027
DT Accuracy: 0.9527687296416938


In [ ]:
# XgBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(objective="binary:logistic", learning_rate = 0.001, max_depth = 10, n_estimators = 100)
xgb.fit(X_train, y_train)
xgb_acc = accuracy_score(y_test, xgb.predict(X_test))
print(f"Training Accuracy of XGB is {accuracy_score(y_train, xgb.predict(X_train))}")
print(f"Testing Accuracy of XGB is {accuracy_score(y_test, xgb.predict(X_test))}")

Training Accuracy of XGB is 0.9236765561372892
Testing Accuracy of XGB is 0.9223669923995657
